**Module 2: Edge-based Segmentation**

* **Theoretical Deep Dive:**
    1.  How do image derivatives relate to edge detection? Explain the concept of gradient magnitude and direction.
    2.  Differentiate between first-order (e.g., Sobel, Prewitt, Roberts) and second-order (e.g., Laplacian, LoG) edge detectors. What are the advantages and disadvantages of each?
    3.  Explain the four main steps of the Canny edge detection algorithm: Noise Reduction, Gradient Calculation, Non-maximum Suppression, and Hysteresis Thresholding. Why is Canny often considered an "optimal" edge detector?
    4.  What are the limitations of purely edge-based segmentation methods for forming complete regions?

* **Practical Implementation/Application:**
    1.  **Basic Edge Detectors:**
        * Load a grayscale image.
        * Apply Sobel, Prewitt, and Roberts operators (use `cv2.filter2D` with custom kernels if `cv2.Sobel` isn't enough, or directly `cv2.Sobel`). Display the resulting edge maps.
        * Calculate and display the gradient magnitude and direction images for one of these operators.
    2.  **Laplacian and LoG:**
        * Apply the Laplacian operator (`cv2.Laplacian`) to an image.
        * Apply Gaussian blur (`cv2.GaussianBlur`) followed by Laplacian (LoG). Experiment with different Gaussian kernel sizes. Display and compare the results.
    3.  **Canny Edge Detector:**
        * Apply `cv2.Canny` to an image.
        * **Experiment:** Systematically vary the `low_threshold` and `high_threshold` values and observe their impact on the detected edges. Pay attention to how the hysteresis linking works.
        * **Challenge (Conceptual Implementation):** For non-maximum suppression, describe the algorithm in pseudocode. For hysteresis thresholding, describe how it uses the two thresholds to connect weak edges to strong ones.

* **Analytical Challenges:**
    1.  **Comparison:** Compare the edge maps produced by Sobel, Laplacian, and Canny. Discuss their sensitivity to noise, edge thickness, and continuity.
    2.  **Segmentation Challenge:** Take an image where you *only* apply an edge detector. Can you directly segment objects based solely on these edges? What are the missing pieces?
    3.  **Preprocessing Impact:** How does applying a Gaussian blur *before* edge detection (as in Canny or LoG) affect the results? Why is this a common pre-processing step?

